In [1]:
import pandas as pd
import sqlite3
import matplotlib
%matplotlib inline

### Creating Helper Functions

In [2]:
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT * FROM sqlite_master 
    WHERE type IN ("table","view");
    '''
    df = run_query(q)
    return df[['name','type']]


#run_query('DROP VIEW view_name;')


In [3]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


### Selecting Albums to Purchase
Write a query that returns each genre, with the number of tracks sold in absolute numbers and in percentages.

In [9]:
#run_command('DROP VIEW genre_quant;')

In [10]:
c = '''
CREATE VIEW genre_quant AS
SELECT il.track_id, il.quantity, t.genre_id, g.name, i.billing_country
FROM invoice_line il
INNER JOIN track t on t.track_id = il.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
INNER JOIN invoice i on i.invoice_id = il.invoice_id
'''
run_command(c)

In [11]:
q = '''
WITH usa_tracks AS
(
SELECT name,
       quantity,
       billing_country
FROM genre_quant
WHERE billing_country="USA"
)
SELECT name genre,
       sum(quantity) total_sold,
       CAST(sum(quantity) AS float)/
       (SELECT sum(quantity)
       FROM genre_quant 
       WHERE billing_country="USA") AS percentage
FROM usa_tracks
GROUP BY genre
ORDER BY total_sold DESC
'''
df = run_query(q)
df.head(30)

,genre,total_sold,percentage
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


###  Analyzing Employee Sales Performance